## Tables to create

Key tables
* alignment
* death
* roles
* settings

Data tables
* player (just player name & id)
* game (general stats about each game)
* gms (gms for each game)
* playergame (each player's stats in each game)
* player_roles (roles for a row in playergame)

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# KEY TABLES

alignment = pd.read_csv("data/keys/alignment.csv").reset_index().rename(columns={"index": "alignment_id"})
print(alignment.head())

setting = pd.read_csv("data/keys/setting.csv")
print(setting.head())

death = pd.read_csv("data/keys/death.csv").reset_index().rename(columns={"index": "death_id"})
death.head()

   alignment_id alignment_char                     alignment_desc  is_elim  \
0             0              G                               Good    False   
1             1              E  Evil (Prim) (team kill / no conv)     True   
2             2              S             Secondary Evil Faction     True   
3             3              T              Tertiary Evil Faction     True   
4             4              ?            Evil Converter (w/kill)     True   

   is_evil  has_kill  has_convert  was_converted  
0    False     False        False          False  
1     True      True        False          False  
2     True      True        False          False  
3     True      True        False          False  
4     True      True         True          False  
         setting  is_sanderson  is_cosmere
0  Cross-Cosmere          True        True
1            Sel          True        True
2       Scadrial          True        True
3        Nalthis          True        True
4         

,death_id,death_char,death_desc,dying_alignment,killer_alignment,non-kills
0,0,S,Survived,none,none,NaN
1,1,E,Killed by Eliminators,not E,E,NaN
2,2,X,Exed,any,none,NaN
3,3,V,Elim killed by Village,E,V,NaN
4,4,F,Villager Killed by Village (Friendly Fire),V,V,NaN


In [3]:
# IMPORTING DATA TAB
data = pd.read_csv("data/original_data/orig_data_040622.csv")

data.rename(inplace=True, columns={
    "ID": "pg_id",
    "Player": "player_name",
    "Alignment": "alignment_char",
    "Faction Outcome": "win", 
    "First Hit": "first_hit", 
    "Last Hit": "last_hit",
    "# of Hits": "num_hits",
    "Death/\n Survival": "death_char", # ???
    "Inactive": "inactive",
    "Mod": "IM",
    "Game Type": "game_format",
    "Game #": "game_num",
    "Unnamed: 15": "game_string", # these depend on other cols. name in sheet pls
    "Role": "role",
    "Unnamed: 18": "secondary_role",
    "Main?": "main_gm"
    })

data = data.filter([
    "pg_id", 
    "player_name",
    "alignment_char",
    "win",
    "first_hit", "last_hit", "num_hits",
    "death_char",
    "inactive",
    "GM", "main_gm", "IM", # "Spec",
    "game_string", #"game_format", "game_num",
    "role", "secondary_role"
    ])

data.head()

# more cleaning here? 

,pg_id,player_name,alignment_char,win,first_hit,last_hit,num_hits,death_char,inactive,GM,main_gm,IM,game_string,role,secondary_role
0,1,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,LG1,GM,Roles that are unique and don't fit
1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG1,Tineye,MR1/LG8/MR7 need work
2,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Cop,NaN
3,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Regular,NaN
4,5,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG1,Smoker,NaN


In [4]:
# IMPORTING GAME TAB

# header = 1 because there's an extra header row on top w categories
game = pd.read_csv("data/original_data/orig_game_040622.csv", header=1)

# cols wanted: id, format, number, string, anon num,
# mech balance, dist balance, IM id, start date, end date
# num cycles, num posts, setting id
# complexity, fundamentals, role madness

# cols still needed: mech balance, dist balance, 
# start date, end date

game.reset_index(inplace=True)

game.columns

Index(['index', 'format', 'num', 'auto?', '# Players', '# Cycles', '# Posts',
       'Fundamentals', 'Winner', 'Complexity', 'Role Madness', 'Anon', 'Mech',
       'Dist', 'Broken', '# Elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       '# Survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'Title/Link', 'Sanderson? ', 'World', 'start_date',
       'end_date', 'link'],
      dtype='object')

In [5]:

# ! Need to be careful about sheet renaming cols - some kind of check?
game.rename(inplace=True, columns={
    "index": "game_id",
    "format": "game_format",
    "num": "game_num", 
    "auto?": "game_string", # optional? 
    #"# Players": "num_players", # optional
    "# Cycles": "num_cycles",
    "# Posts": "num_posts",
    "Fundamentals": "fundamentals", 
    "Role Madness": "role_madness",
    #"Winner": "winner", # optional
    "Complexity": "complexity",
    #"Broken": "broken", # TODO make into mech/dist once that's set in sheet
    "Mech": "mechanics_balance", # blank rn!!
    "Dist": "distribution_balance",
    "World": "setting", # need to make setting table and replace this w setting_id
    "Anon": "anon_num", 
    "Title/Link": "title", # need to get link separately. hidden col? 
    # LINK
    "start_date": "start_date",
    "end_date": "end_date"
    })

print(game.columns)

# remaining columns are just calculated from data test
game = game.filter([
    "game_id",
    "game_format", "game_num", "game_string",
    "num_players",
    #"IM", # currently blank - pulling from Data instead
    "num_cycles", 
    "num_posts", 
    "fundamentals", "role_madness", "complexity", 
    "winner", 
    "mechanics_balance", "distribution_balance",
    #"broken", 
    "setting",
    "anon_num",
    "title",
    "start_date", "end_date"
])


game.tail()
# check for almost all NaN rows! extra checkboxes cause this

Index(['game_id', 'game_format', 'game_num', 'game_string', '# Players',
       'num_cycles', 'num_posts', 'fundamentals', 'Winner', 'complexity',
       'role_madness', 'anon_num', 'mechanics_balance', 'distribution_balance',
       'Broken', '# Elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       '# Survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'title', 'Sanderson? ', 'setting', 'start_date',
       'end_date', 'link'],
      dtype='object')


,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date
208,208,LG,83.0,LG83,6,NaN,V/E,False,NaN,NaN,NaN,NaN,NaN,The Survivor,NaN,NaN
209,209,BT,1.0,BT1,4,NaN,V/E,False,NaN,NaN,NaN,Cross-Cosmere,NaN,Auction of Stories,NaN,NaN
210,210,QF,59.0,QF59,7,NaN,V/E,False,NaN,NaN,NaN,Roshar,NaN,Bachelor - Roshar Edition!,NaN,NaN
211,211,MR,57.0,MR57,NaN,NaN,V/E,False,Semi Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,212,LG,84.0,LG84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


END CLEANING 

# SQL table making

In [6]:
# PLAYER - DONE
unique_players = data["player_name"].unique()

player = pd.DataFrame({"player_id": range(len(unique_players)), "player_name": unique_players})

player.head()


,player_id,player_name
0,0,Meta
1,1,Cracknut
2,2,Edgedancer
3,3,Mailliw73
4,4,Aether


In [7]:
# SETTINGS - DONE

unique_worlds = game["setting"].unique()

new_settings = []

# for each loc not there, put down as non-Sanderson and print that you're doing so
for world in unique_worlds:
    if world not in setting["setting"].unique():
        if (str(world) == "nan"): # don't want to insert NaN worlds
            continue # check
        new_settings.append({"setting": world, "is_sanderson": False, "is_cosmere": False})
        print(f"inserted {world}")

new_settings_df = pd.DataFrame(new_settings)

setting = pd.concat([setting, new_settings_df], ignore_index=True).reset_index().rename(columns={"index": "setting_id"})

setting.tail()


inserted Hunger Games
inserted Game of Thrones
inserted KKC
inserted Death Note
inserted Codex Alera
inserted Pokemon
inserted Black Prism
inserted Red Rising
inserted Discworld
inserted Elder Scrolls
inserted My Little Pony
inserted Les Miserables
inserted Gunnerkrigg Court
inserted Lord of the Rings
inserted Kingkiller Chronicles
inserted Princess Bride
inserted Zelda
inserted Firefly


,setting_id,setting,is_sanderson,is_cosmere
29,29,Lord of the Rings,False,False
30,30,Kingkiller Chronicles,False,False
31,31,Princess Bride,False,False
32,32,Zelda,False,False
33,33,Firefly,False,False


In [8]:
# ALIGNMENT / DEATH
# do the same check as above with unique death and alignment tables, but don't insert by default - just print thing

al = data["alignment_char"].unique()

for a in al:
    if a not in alignment["alignment_char"].unique():
        print(a)



de = data["death_char"].unique()

for d in de:
    if d not in death["death_char"].unique():
        print(d)


# TODO fix rows with these things (or could do in SQL insertion or merge)
# currently I think merge just has them as NaN so SQL makes them null? which works 

nan
nan
L?
?
V/M
L
V/E
X/E


In [9]:
# TODO id replacements:
# in PG - player_id, game_id, alignment_id, death_id

s = pd.Series(player.set_index("player_name")["player_id"])
data["player_id"] = data["player_name"].map(s)

s = pd.Series(game.set_index("game_string")["game_id"])
data["game_id"] = data["game_string"].map(s)

s = pd.Series(alignment.set_index("alignment_char")["alignment_id"])
data["alignment_id"] = data["alignment_char"].map(s)

s = pd.Series(death.set_index("death_char")["death_id"])
data["death_id"] = data["death_char"].map(s)

data.head()


,pg_id,player_name,alignment_char,win,first_hit,last_hit,num_hits,death_char,inactive,GM,main_gm,IM,game_string,role,secondary_role,player_id,game_id,alignment_id,death_id
0,1,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,LG1,GM,Roles that are unique and don't fit,0,0.0,NaN,NaN
1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG1,Tineye,MR1/LG8/MR7 need work,1,0.0,0.0,1.0
2,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Cop,NaN,2,0.0,0.0,2.0
3,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Regular,NaN,3,0.0,0.0,2.0
4,5,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG1,Smoker,NaN,4,0.0,1.0,3.0


In [10]:
s = pd.Series(setting.set_index("setting")["setting_id"])
g = game["setting"].map(s)
game["setting_id"] = g.copy() # copy might not be necessary?
game.head()

,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date,setting_id
0,0,LG,1.0,LG1,4,462.0,V/E,False,Standard,NaN,NaN,Scadrial,NaN,In the Wake of Koloss,NaN,NaN,2.0
1,1,LG,2.0,LG2,11,1232.0,CON,False,Standard,NaN,NaN,Scadrial,NaN,Devil's Den,NaN,NaN,2.0
2,2,LG,3.0,LG3,7,739.0,V/E,False,Standard,NaN,NaN,Scadrial,NaN,Blackwater Village,NaN,NaN,2.0
3,3,LG,4.0,LG4,9,1029.0,V/E,False,Semi Standard,NaN,NaN,Nalthis,NaN,Colours of War,NaN,NaN,3.0
4,4,QF,1.0,QF1,4,118.0,V/E,False,Basic,NaN,NaN,Sel,NaN,Jeskeri Mysteries,NaN,NaN,1.0


In [11]:
# Pull rest of stuff into game table from PG: IM id

# because apparently Wilson was 'backup IM' for Orlok for QF38. DEFINITELY change this to be more robust
IM_list = data[data["IM"] == "Y"].filter(["player_id", "game_id"]).drop(4313)
#IM_list["game_id"].astype("int", copy=False)

game = game.merge(IM_list, on="game_id", how="left").rename(columns={"player_id": "IM_id"})

game.tail()


,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date,setting_id,IM_id
208,208,LG,83.0,LG83,6,NaN,V/E,False,NaN,NaN,NaN,NaN,NaN,The Survivor,NaN,NaN,NaN,235.0
209,209,BT,1.0,BT1,4,NaN,V/E,False,NaN,NaN,NaN,Cross-Cosmere,NaN,Auction of Stories,NaN,NaN,0.0,235.0
210,210,QF,59.0,QF59,7,NaN,V/E,False,NaN,NaN,NaN,Roshar,NaN,Bachelor - Roshar Edition!,NaN,NaN,4.0,64.0
211,211,MR,57.0,MR57,NaN,NaN,V/E,False,Semi Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,235.0
212,212,LG,84.0,LG84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.0


In [12]:
game_final = game.filter([
    "game_id", "game_format", "game_num", "game_string",
    "anon_num", #"mechanics_balance", "distribution_balance",
    "IM_id", "num_cycles", "num_posts", #"start_date", "end_date",
    "title", "setting_id",
    "complexity", "fundamentals", "role_madness"
    ])

# null / nan works just fine for these
# game_final["IM_id"] = game_final["IM_id"].fillna(None) # doesn't work
# game_final["num_cycles"] = game_final["num_cycles"].fillna(-1)
# game_final["num_posts"] = game_final["num_posts"].fillna(-1)


game_final.head()


,game_id,game_format,game_num,game_string,anon_num,IM_id,num_cycles,num_posts,title,setting_id,complexity,fundamentals,role_madness
0,0,LG,1.0,LG1,NaN,NaN,4,462.0,In the Wake of Koloss,2.0,Standard,V/E,False
1,1,LG,2.0,LG2,NaN,NaN,11,1232.0,Devil's Den,2.0,Standard,CON,False
2,2,LG,3.0,LG3,NaN,NaN,7,739.0,Blackwater Village,2.0,Standard,V/E,False
3,3,LG,4.0,LG4,NaN,NaN,9,1029.0,Colours of War,3.0,Semi Standard,V/E,False
4,4,QF,1.0,QF1,NaN,NaN,4,118.0,Jeskeri Mysteries,1.0,Basic,V/E,False


In [13]:
# make GMs table from PG

gms = data[data["GM"] == "Y"].filter(["player_id", "game_id", "main_gm"])

#gms.rename(columns={"player_id": "gm_id"}, inplace=True) # not how SQL table is rn. Maybe should be? 

gms["main_gm"] = gms["main_gm"].map({"Y": True, "nan": False, np.nan: False})

gms.head()


,player_id,game_id,main_gm
0,0,0.0,True
20,0,1.0,True
62,15,2.0,True
83,5,3.0,True
135,48,4.0,True


In [14]:
# if roles are in, make roles table here

In [15]:
# clean PG 


playergame = data[data["GM"] != "Y"]
playergame = playergame[playergame["IM"] != "Y"]
# playergame = playergame[playergame["Spec"] != "Y"] # spec is filtered out at start atm


playergame["win"] = playergame["win"].map({"W": True, "L": False, "D": np.nan})
playergame["inactive"] = playergame["inactive"].map({"Y": True, "NaN": False, np.nan: False})

playergame = playergame.filter(['pg_id', 'player_id', 'game_id', 'alignment_id','death_id', 'first_hit', 'last_hit', 'num_hits', 'win', 'inactive' ])

#playergame = playergame.convert_dtypes() #nvm do not

playergame.head()


,pg_id,player_id,game_id,alignment_id,death_id,first_hit,last_hit,num_hits,win,inactive
1,2,1,0.0,0.0,1.0,1.0,1.0,1.0,False,False
2,3,2,0.0,0.0,2.0,2.0,2.0,1.0,False,False
3,4,3,0.0,0.0,2.0,4.0,4.0,1.0,False,False
4,5,4,0.0,1.0,3.0,4.0,4.0,1.0,True,False
5,6,5,0.0,1.0,0.0,NaN,NaN,0.0,True,False


In [16]:
# OPTIONAL: CSV EXPORTS 
# player.to_csv("data/player.csv", index=False)
# playergame, game, alignment, death, blah blah blah

In [17]:
# removing bad characters

# player["player_name"] = player["player_name"].str.replace("'", "") 

# game["title"] = game["title"].str.replace("'", "")

# removes ? in MR7. Definitely need to find a way to do typing better though
game_final["num_cycles"] = game_final["num_cycles"].str.replace("?", "")

game_final.iloc[25:30]
# player.iloc[290:310]


/tmp/ipykernel_951804/400110520.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  game_final["num_cycles"] = game_final["num_cycles"].str.replace("?", "")


,game_id,game_format,game_num,game_string,anon_num,IM_id,num_cycles,num_posts,title,setting_id,complexity,fundamentals,role_madness
25,25,LG,12.0,LG12,NaN,6.0,9,1301.0,Shadows of Elantris,1.0,Complex,EE,False
26,26,QF,8.0,QF8,NaN,11.0,11,529.0,The Steelway Express,2.0,Standard,V/E,False
27,27,MR,7.0,MR7,NaN,6.0,12,665.0,Eighty Splendid Suns,1.0,Faction/FFA,FAC,False
28,28,LG,13.0,LG13,NaN,11.0,8,607.0,Traitors on the Plains,4.0,Standard,V/E,False
29,29,QF,9.0,QF9,NaN,11.0,6,271.0,The Court of the Gods,3.0,Basic,V/E,False


## Tracking SQL appropriateness: 
* player: DONE
* setting: DONE  (remove nan row)
* alignment: maybe? 
* death: DONE (have extra cols not being inserted)
* **game**: nooo (missing some cols; bad types - see below)
* gms: DONE ish (not currently marking main GM bool, but SQL has a default and idc)
* **playergame**: nope (below)

### Problems with game 
* Missing columns: 
    - Mech and dist balance (not yet in sheet; just make all null? )
    - start and end date (not yet in sheet)
    - link (not in sheet in a helpful way)
* Type problems: 
    - num_posts is a float (??)

### Problems with playergame 
* Type problems: 
    - first, last, and num hits are all floats and should be ints
    - ids for game, alignment, and death should be ints
* pinchhitter doesn't exist

### Where to check for bad characters:
* game.title 
* player.player_name
* game.fundamentals?

### Other
* PG needs to be cleaned to remove things not in death / alignment tables; want to be able to do constraints also on complexity / etc 

At this point should have the tables:
* playergame
* games (game)
* player
* gms
* alignment
* death
* settings

time for...
## SQL Insertion


In [18]:
import psycopg2
db = 1

In [19]:
if db != 0:
    cnx = psycopg2.connect(user="isanford_123", database="isanford", host="localhost", password="12345678")
    cursor = cnx.cursor()


In [20]:
def insertion_maker(db, db_name): # not sure name is necessary
    ins_base = f"insert into {db_name} ("
    ins_cols = ",".join(list(db.columns)) 

    ins = f"{ins_base}{ins_cols}) values (" 

    vals = ",".join(["%s" for col in db.columns])

    ins += vals + ");"

    return ins


In [21]:
def insert_table(db, db_name, cursor = 0):
    base = insertion_maker(db, db_name)

    err_ct = 0

    for row in db.iterrows():
        row_data = [r if str(r) != "nan" else None for r in row[1]]

        if (cursor != 0):
            try:
                cursor.execute(base, row_data)
            except: # how do I also make it print the error
                print(base)
                print(row_data) 
                err_ct += 1
                print(row[0])
                return # for now

        else:
            print(row_data)
    
    print(f"{err_ct} errors inserting data into {db_name}.")

In [22]:
def insert_table_err(db, db_name, cursor = 0):

    base = insertion_maker(db, db_name)
    
    print(base)
    
    err_ct = 0

    for row in db.iterrows():
        row_data = [r if str(r) != "nan" else None for r in row[1]]

        if (cursor != 0):
            cursor.execute(base, row_data)
        else:
            print(row_data)
            
    
    print(f"{err_ct} errors inserting data into {db_name}.")

In [23]:
# NOTE: insert_table REQUIRES that every column in table is set to go into db under same name

insert_table(player, "player", cursor)
insert_table(setting, "setting", cursor)
insert_table(alignment, "alignment", cursor)
insert_table(death.filter(["death_id", "death_char", "death_desc"]), "death", cursor) # dubious
insert_table(game_final, "game", cursor)
insert_table(gms, "gms", cursor)
insert_table(playergame, "playergame", cursor)
# (roles)

cnx.commit() # brave

0 errors inserting data into player.
0 errors inserting data into setting.
0 errors inserting data into alignment.
0 errors inserting data into death.
0 errors inserting data into game.
0 errors inserting data into gms.
0 errors inserting data into playergame.


In [24]:
cursor.close()
cnx.close()

END (hopefully)